In [1]:
pip install timm

     |████████████████████████████████| 431 kB 4.0 MB/s 


In [2]:
import os
import numpy as np
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
import matplotlib.pyplot as plt
from skimage import io, transform
from torchvision import transforms, utils
import timm
import torch.nn as nn
from sklearn.metrics import accuracy_score
import time

device = torch.device('cuda')
device

device(type='cuda')

In [3]:
class CustomImageDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, mode='train'):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform
    self.mode = mode

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = read_image(img_path)
    if self.mode == 'train':
      label = self.img_labels.iloc[idx, 1]
    else:  # test set has no label
      label = 'temp'
    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      label = self.target_transform(label)
    return image, label

In [4]:
label_csv = '/content/drive/MyDrive/ZINDI-image/csv_files/Test.csv'
test_dir = '/content/drive/MyDrive/ZINDI-image/test'

test_ds = CustomImageDataset(annotations_file=label_csv,
                             img_dir=test_dir, 
                             transform=transforms.Compose([
                                                           transforms.ToPILImage(),
                                                           transforms.Resize((224,224)),
                                                           transforms.ToTensor(),
                                                           transforms.Normalize([0.44280738, 0.5332802, 0.33625707], 
                                                                                [0.17611444, 0.1734216, 0.17963623])
                                                           ]),
                             mode = 'test'
                             )
test_dataloader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4)

In [5]:
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
        
  def forward(self, x):
    x = self.model(x)
    return x

model = Network().to(device)
# print(model)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


In [6]:
# Loading the model
model.load_state_dict(torch.load('/content/drive/MyDrive/ZINDI-image/notebooks/PT/saved/saved_model.pth'))
model.eval()

Network(
  (model): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, tr

In [7]:
pred_list = []  # to store the prediction

with torch.no_grad():
  for data, _ in test_dataloader:
    if torch.cuda.is_available():
      data = data.cuda()
    with torch.cuda.amp.autocast():
      pred = model(data)
    pred_list.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [8]:
submission = pd.read_csv('/content/drive/MyDrive/ZINDI-image/csv_files/SampleSubmission.csv')

submission['Label'] = pred_list

submission

,Image_id,Label
0,id_00exusbkgzw1b.jpg,0
1,id_03dqinf6w0znv.jpg,0
2,id_046yl0cxn3ybz.jpg,1
3,id_04athdtx2abyg.jpg,0
4,id_062aauf9e9jk0.jpg,0
...,...,...
1075,id_zv5fvjnakvf1r.jpg,1
1076,id_zvpikh1z30arn.jpg,0
1077,id_zypilwkudljyz.jpg,0
1078,id_zz9lwehh5sxdp.jpg,1


In [10]:
submission.to_csv('/content/drive/MyDrive/ZINDI-image/csv_files/PT_effnetb0.csv', index = False)